In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

np.random.seed(1234)

In [3]:
print(torch.cuda.get_device_name(torch.cuda.current_device()))  # GPU name

# CUDA support
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

NVIDIA GeForce RTX 2060
Using device: cuda


In [22]:
p = 20
bc_type = 3

x = torch.linspace(0, 1, p).view(-1, 1)
fixed = torch.ones(p) * 0
free = torch.ones(p) * 1
pinned = torch.ones(p) * 2

v_fixed = nn.functional.one_hot(fixed.to(torch.int64), num_classes=bc_type).to(torch.float32)
v_free = nn.functional.one_hot(free.to(torch.int64), num_classes=bc_type).to(torch.float32)
v_pinned = nn.functional.one_hot(pinned.to(torch.int64), num_classes=bc_type).to(torch.float32)




b1 = torch.cat((x, v_fixed, v_free), dim=1)
b2 = torch.cat((x, v_pinned, v_fixed), dim=1)
b3 = torch.cat((x, v_free, v_pinned), dim=1)
b4 = torch.cat((x, v_fixed, v_pinned), dim=1)
b5 = torch.cat((x, v_pinned, v_free), dim=1)
b6 = torch.cat((x, v_free, v_fixed), dim=1)

In [23]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class BeamDataset(Dataset):
    def __init__(self, num_samples_per_beam=100, beam_configs=None):
        """
        Args:
            num_samples_per_beam (int): Number of training points per beam configuration.
            beam_configs (list of dicts): Each dictionary contains the parameters of a beam problem.
        """
        self.data = []
        for config in beam_configs:
            x = torch.linspace(0, 1, num_samples_per_beam).view(-1, 1)  # Sampled x positions (normalized)
            p_bc = torch.tensor(config['p_bc'], dtype=torch.float32).repeat(num_samples_per_beam, 1)  # One-hot BCs
            w_true = torch.tensor(config['w'], dtype=torch.float32).view(-1, 1)  # Ground truth displacement

            self.data.append(torch.cat((x, p_bc, w_true), dim=1))  # Stack into a single tensor

        self.data = torch.cat(self.data, dim=0)  # Merge all beam configurations into one dataset

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        sample = self.data[idx]
        x = sample[0].unsqueeze(0)  # x position
        p_bc = sample[1:-1]  # Boundary condition encoding
        w_true = sample[-1]  # True displacement (optional, if supervised)
        return x, p_bc, w_true


In [24]:
# Define different beam configurations
beam_configs = [
    {'p_bc': [1, 0, 0], 'w': np.sin(np.linspace(0, np.pi, 100))},  # Fixed-free beam
    {'p_bc': [0, 1, 0], 'w': np.linspace(0, 0, 100)},  # Pinned-pinned beam
    {'p_bc': [0, 0, 1], 'w': np.cos(np.linspace(0, np.pi, 100))}   # Clamped-clamped beam
]

# Create dataset and DataLoader
dataset = BeamDataset(num_samples_per_beam=100, beam_configs=beam_configs)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Example batch retrieval
for x_batch, p_bc_batch, w_batch in dataloader:
    print("x:", x_batch.shape)  # (batch_size, 1)
    print("p_bc:", p_bc_batch.shape)  # (batch_size, 3) - One-hot BCs
    print("w:", w_batch.shape)  # (batch_size, 1)
    break


x: torch.Size([32, 1])
p_bc: torch.Size([32, 3])
w: torch.Size([32])


In [25]:
import torch.nn as nn

class PINN(nn.Module):
    def __init__(self, input_size, hidden_size=50, output_size=1):
        super(PINN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x, p_bc):
        inputs = torch.cat((x, p_bc), dim=1)  # Concatenate x and boundary condition encoding
        return self.network(inputs)


In [26]:
import torch.optim as optim

# Initialize model, loss, and optimizer
model = PINN(input_size=4)  # x (1D) + 3 BC encoding = 4 input features
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 1000

for epoch in range(num_epochs):
    for x_batch, p_bc_batch, w_batch in dataloader:
        optimizer.zero_grad()
        w_pred = model(x_batch, p_bc_batch)
        loss = criterion(w_pred, w_batch)
        loss.backward()
        optimizer.step()
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.6f}")


c:\Users\alexa\miniconda3\envs\pinn\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\alexa\miniconda3\envs\pinn\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([12])) that is different to the input size (torch.Size([12, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0: Loss = 0.420033
Epoch 100: Loss = 0.188474
Epoch 200: Loss = 0.380477
Epoch 300: Loss = 0.260603
Epoch 400: Loss = 0.534071
Epoch 500: Loss = 0.292803
Epoch 600: Loss = 0.239935
Epoch 700: Loss = 0.327213
Epoch 800: Loss = 0.178003
Epoch 900: Loss = 0.259345


In [ ]:
class DNN(nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        modules = []
        for i in range(len(layers) - 2):  # Exclude last layer for activation
            modules.append(nn.Linear(layers[i], layers[i+1]))
            modules.append(nn.Tanh())
        modules.append(nn.Linear(layers[-2], layers[-1]))  # Last layer (no activation)
        self.network = nn.Sequential(*modules)

        for layer in self.network:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)

    def forward(self, x):
        return self.network(x)

In [ ]:
class PINN():
    def __init__(self, layers, x, q, lb, rb):

        self.w_pde = 1.0
        self.w_bc = 1.0

        self.x = x
        self.q = q

        self.lb = lb
        self.rb = rb

        # DNN
        self.dnn = DNN(layers).to(device)

        # Optimizer
        self.optimizer_lbfgs = torch.optim.LBFGS(
            self.dnn.parameters(),
            lr=0.1,
            max_iter=50000,
            max_eval=50000,
            history_size=50,
            tolerance_grad=1e-7,
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe"
        )

        self.optimizer_adam = torch.optim.Adam(self.dnn.parameters(), lr=0.01)
        self.iter = 0

    def model_value(self, x):
        out = self.dnn(x)
        print(out)
        u = out[:,0].view(-1, 1)
        m = out[:,1].view(-1, 1)
        return u, m
    
    def boundary_cond(self, cond, u, u_x, m, m_x):
        bc_loss = 0

        match cond:
            case 'pinned':
                bc_loss += u**2 + m**2
            case 'fixed':
                bc_loss += u**2 + u_x**2
            case 'free':
                bc_loss += m**2 + m_x**2
            case 'roller':
                bc_loss += u_x**2 + m_x**2
        return bc_loss

    def loss_func(self, x):
        u, m = self.model_value(x)
        u_x = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]
        u_2x = torch.autograd.grad(u_x, x, torch.ones_like(u_x), create_graph=True)[0]

        m_x = torch.autograd.grad(m, x, torch.ones_like(m), create_graph=True)[0]
        m_2x = torch.autograd.grad(m_x, x, torch.ones_like(m_x), create_graph=True)[0]

        # BC
        bc_loss = self.boundary_cond(self.lb, u[0], u_x[0], m[0], m_x[0])
        bc_loss += self.boundary_cond(self.rb, u[-1], u_x[-1], m[-1], m_x[-1])

        # PDE
        pde_loss = torch.mean(torch.pow(m_2x + self.q, 2))
        pde_loss += torch.mean(torch.pow(u_2x + m, 2))

        return pde_loss, bc_loss


    def lbfgs_func(self):
        pde_loss, bc_loss = self.loss_func(self.x)
        loss = self.w_pde*pde_loss + self.w_bc*bc_loss

        self.optimizer_lbfgs.zero_grad()
        loss.backward()

        if self.iter % 100 == 0:
            print(f"Iter: {self.iter}, Loss: {'{:e}'.format(loss.item())}")
            print(f"PDE: {'{:e}'.format(pde_loss.item())}, BC: {'{:e}'.format(bc_loss.item())}")
        self.iter += 1
        return loss
    
    def train(self, epochs=1000):
        self.dnn.train()
        for epoch in range(epochs):
            pde_loss, bc_loss = self.loss_func(self.x)
            loss = self.w_pde*pde_loss + self.w_bc*bc_loss

            self.optimizer_adam.zero_grad()
            loss.backward()
            self.optimizer_adam.step()

            if epoch % 200 == 0:
                print(f"Epoch: {epoch}, Loss: {'{:e}'.format(loss.item())}")
                print(f"PDE: {'{:e}'.format(pde_loss.item())}, BC: {'{:e}'.format(bc_loss.item())}")
        # self.optimizer_lbfgs.step(self.lbfgs_func)

    def predict(self, X):
        x = torch.tensor(X, requires_grad=True).float().to(device)

        self.dnn.eval()
        u_c, m_c = self.model_value(x)
        u_c = u_c.detach().cpu().numpy()
        m_c = m_c.detach().cpu().numpy()
        return u_c, m_c